# The Kalman Filter

introduced in 1958, is one of the most used filters to process noisy data.

In order to be able to understand its functionality, the following text shows an overview of the theory and the required computations. Since this notebook aims to give a very small and basic overview, we will not focus on the proofs of the Kalman Filter math. For more detailed explanations with deeper insights on the Kalman Filter kindly refer to [[1]](#Thrun) and [[2]](#kf_notebook).


The basic functionality of the Kalman Filter is quite simple as we have only two stages:
1. Predict and
2. Update


Within the predict part, we estimate a potential position using a motion model and whithin the update part, we take in the observations, identify how confident these are based on our expected estimate from the predictions part and finally update the estimate.

The Kalman Filter is often mentioned to be a "Gaussian Filter"; which basically says the following:

1. Our inputs are Gaussian distributed
2. All computations within the Kalman Filter preseve the Gaussian distribution
   
The second describes that adding and multiplying two Gaussians $\mathcal{N}(\mu_1,\,\sigma_1^{2})$ and $\mathcal{N}(\mu_2,\,\sigma_2^{2})$, is based on the following rules.

**Addition** 
$$
\mu = \mu_1 + \mu_2 
$$

$$
\sigma^{2} = \sigma_1^{2} + \sigma_2^{2} 
$$


**Multiplication**

$$
\mu = \frac{\mu_1 \sigma_2^{2} \ + \mu_2 \sigma_1^{2}}{\sigma_1^{2} + \sigma_2^{2}}
$$

$$
\sigma^{2} = \frac{\sigma_1^{2} \sigma_2^{2}}{\sigma_1^{2} + \sigma_2^{2}} 
$$


## Kalman Filter Theory

Let's go trough some theory first.

As previously mentioned, the Kalman Filter contains two steps: **predict** and **update**. We will start with an overview of the first followed by the latter.

The values required to be calculated within both steps are

```
Predict
x: state estimate (mean of Gaussian)
P: state estimate covariance
Q: process covariance matrix 
B: control input model
u: control input
F: state transition matrix

Update
z: measurement
H: measurement function
R: measurement noise
y: error
S: innovation
K: Kalman Gain
```

### Prediction

Out state vector **x** consists of two elements, the actual $x$ position and the velocity $v$, that represents the hidden or latent variable as it is something we do not measure but derive from the measurements.
$$
\bf{x} = \begin{bmatrix} x \\ v \end{bmatrix}
$$

Yes, the `state` $\bf{x}$ and the `observation` $x$ are (unfortunately) represented by the same variable (while the state is expressed in bold).

The `state covariance` is the variance of a Gaussian distribution in multidimensional space. In simple words: it represents the confidence of an estimated state (a higher covariance implies a greater spread).
This matrix is constantly updated but requires initialisation. We will set the $x$ position covariance to $10$ and the velocity to $5$, implying
$$
P = \begin{bmatrix} 10 & 0 \\ 0 & 5 \end{bmatrix}
$$

The `process covariance matrix` $\bf{Q}$ represents the uncertainty of our prediction model. Since we expect that our prediction can never be 100% accurate, we can model the deviation using this matrix. $\bf{Q}$ has also a very practical benefit: it enables the `state covariance` $\bf{P}$ to be positive definite ($\neq 0$). A zero state covariance simply states that we are too confident with our predictions (hence, no covariance), which in general is a bit too optimistic. It moreover implies that the `innovation` (explained below) would become a zero matrix as well (making it impossible to calculate its inverse). 

We will set $\bf{Q}$ to



$$
\bf{Q} = \begin{bmatrix} 0.05 \\ 0.05 \end{bmatrix}
$$


The `control input` and the `control model` ($\bf{u}$ and $\bf{B}$) aim to define the uncertainty resulting from a control system. In our case this would be subject to the vehicles electrical / mechanical controllers that transform a digital signal (for steering, acceleration / deceleration) into mechanical motion. We will omit modelling the control system uncertainty (as it is subject to very detailed Kalman Filter design requiring knowlegde of the control system such as the power train system used).

<br/>

The `transition matrix` $\bf{F}$ is the interesting one. It defines how states are predicted at step $i$ using the previous state estimate.

It could become arbitray complex and in our case we will keep it a bit simple by using the **constant velocity motion model** that implies a constant velocity between two observation steps. In other words: we will expect an object to travel with the same speed in between two actual observations. Refer `01_motion_models` notebook for more details on the model used.
Hence, we define $\bf{F}$ as 
$$
\bf{F} = \begin{bmatrix} 1 & \Delta t \\ 0 & 1 \end{bmatrix}
$$

Now, for the **prediction step**, we need to estimate our state as well as our covariance. 

For the state estimation at at prediction step $i$ we multiply the transition matrix $F$ with the previous state $\bf{x_{i-1}}$ and add the control input / model to it, yielding

$$
\bf{x_i} = F \cdot \bf{x_{i-1}} + B \cdot u
$$

Since we omit the control specific parts, we will define our state estimation as

$$
\bf{x_i} = F \cdot \bf{x_{i-1}}
$$

One may ask "how this could represent a **linear motion model**". The answer reveals itself by having a closer look at the resulting calculations

$$
\bf{x_i} =  F \cdot \bf{x_{i-1}}
= \begin{bmatrix} 1 & \Delta t \\ 0 & 1 \end{bmatrix} \cdot \begin{bmatrix} x_{i-1} \\ v \end{bmatrix}
=  \begin{bmatrix} 1 \cdot x_{i-1}  + \Delta t \cdot v_{i-1}  \\ 0 \cdot x_{i-1}  + 1 \cdot v_{i-1} \end{bmatrix} = \begin{bmatrix} x_{i-1}  + \Delta t \cdot v_{i-1} \\ v_{i-1} \end{bmatrix}
$$

=> for the new position in $x$ direction, we would use $x_{i-1} + \Delta t \cdot v_{i-1}$, which defines a *linear* motion and for the velocity, we would just keep the old value (as we are expecting it to be constant).


<br/>

For the covariance part, we use
$$
\bf{P} = \bf{F} \cdot \bf{P} \cdot \bf{Fˆ{T}} + \bf{Q}
$$

... which looks rather cryptic. Let's have a closer look on that one as well using the intial values defined for $\bf{P}$ above (omitting $\bf{Q}$).

$$
\bf{P_i} = \bf{F} \cdot \bf{P_{i-1}} \cdot \bf{Fˆ{T}}
= \begin{bmatrix} 1 & \Delta t \\ 0 & 1 \end{bmatrix} \cdot \begin{bmatrix} 10 & 0 \\ 0 & 5 \end{bmatrix} \cdot \begin{bmatrix} 1 & 0 \\ \Delta t & 1 \end{bmatrix}
= \begin{bmatrix} 10 & 5 \Delta t \\ 0 & 5 \end{bmatrix} \cdot \begin{bmatrix} 1 & 0 \\ \Delta t & 1 \end{bmatrix}
= \begin{bmatrix} 10 + 5 \Delta tˆ2 & 5 \Delta t \\ 5 \Delta t & 5\end{bmatrix}
$$

We can see that the resulting matrix
$$
\begin{bmatrix} 10 + 5 \Delta tˆ2 & 5 \Delta t \\ 5 \Delta t & 5\end{bmatrix}
$$

shows is a correlation between the observable variable (the position) and the latent variable (the velocity). Although we have initially defined that both are uncorrelated (refer the initialisation of $\bf{P}$), we can see that both variances are affecting each other.

A more detailed explanation on this particular topic is presented in [[2]](#kf_notebook).


### Update

The `measurement` we receive in our 1D case is the position in $x$ direction. Hence, the measurement is a 1D vector

$$
z = \begin{bmatrix} x \end{bmatrix}
$$

The `measurement function` defines which states are hidden and which ones are actually observed. In our case the position $x$ is the variable we observe and velocity $v$ is the one we derive from the measurements. Hence, our measurement function is
$$
\bf{H} = \begin{bmatrix} 1 & 0 \end{bmatrix}
$$

<br/>

The `measurement noise` $\bf{R}$ defines the uncertainty of the measurements and is something we would need to get from the sensor manufacturer. Since this is an information we do not have for or dataset, we set the measurement noise for our sensor to
$$
\bf{R} = 0.05
$$

<br/>

Now, for the `update` step have several equations. We first, need to calculate an error representing the difference between the predicted and the observed value 
$$
y = z - \bf{H} \cdot \bf{x}
$$

... which is straighforward to understand as it is literally just the differce between the estimated state in "measurement space" (= we do not consider the velocity for measurement).

Next ist to calculate the `innovation` $\bf{S}$ 

$$
\bf{S} = \bf{H} \cdot \bf{P} \cdot \bf{Hˆ{T}} + \bf{R}
$$

The purpose of $\bf{S}$ is to "project the covariance matrix into measurement space" [2].

<br/>

Followed by the `Kalman Gain`:

$$
\bf{K} = \bf{P} \cdot \bf{Hˆ{T}} \cdot \bf{S}ˆ{-1} 
$$

that "specifies the degree to which the measurement is incoprated into the new state estimate" [[1]](#Thrun) (and is very similar to a weight).

<br/>

And we finally got to the update of our state estimation as well as the covariance at step $i$

$$
\bf{x_i} = \bf{x_{i-1}} + \bf{K} \cdot \bf{y}
$$

$$
\bf{P_i} = (\bf{I} - \bf{K} \cdot \bf{H}) \cdot \bf{P_{i-1}}
$$


## Kalman Filter Implementation

Once the matrices above are clear, the implementation is rather simple.

In [1]:
import numpy as np
from scipy.linalg import inv

class KalmanFilter(object):
    def __init__(self, P, F, R, Q, H, init_state = np.zeros((2)), num_states = 2):
        self.P = P
        self.F = F
        self.R = R
        self.Q = Q
        self.H = H
        self.num_states = num_states
        self.x = init_state

    def predict(self):
        self.x = self.F @ self.x
        self.P = self.F @ self.P @ self.F.T + self.Q
        
        
    def update(self, z):
        # error
        y = z - self.H @ self.x
        # innovation
        S = self.H @ self.P @ self.H.T + self.R
        # Kalman gain
        K = self.P @ self.H.T @ inv(S)
        # update state and covariance
        self.x = self.x + (K @ y)
        self.P = (np.eye(self.num_states, self.num_states) - K @ self.H) @ self.P

# References


<a id="Thrun">[1]</a>: Thrun, Burgard, Fox, Probabilistic Robotics, MIT Press, 2006

<a id="kf_notebook">[2]</a>: https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python/blob/master/06-Multivariate-Kalman-Filters.ipynb